In [1]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier

In [2]:
df = pd.read_csv('reg_ready.csv')
df['Date'] = pd.to_datetime(df['Date'])
df.head()

,Date,Open,Close,monthly_return,period of downturn,Recessions,INDPRO,yc_date,YIELD_CURVE,UNEMPLOYMENT,BGI_diff_from_2_month_high,KO_diff_from_2_month_high,KR_diff_from_2_month_high,MAR_diff_from_2_month_high,PPG_diff_from_2_month_high,SHW_diff_from_2_month_high,unemployment_diff_from_6_month_low,INDPRO_diff_from_6_month_high
0,1950-02-01,17.049999,17.219999,0.009971,0,0,NaN,1976-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1950-03-01,17.240000,17.290001,0.002900,0,0,14.4279,1976-06-01,NaN,6.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1950-04-01,17.340000,18.070000,0.042099,0,0,14.4833,1976-06-01,NaN,6.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1950-05-01,18.219999,18.780001,0.030736,0,0,14.9541,1976-06-01,NaN,6.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1950-06-01,18.770000,17.690001,-0.057539,0,0,15.4526,1976-06-01,NaN,5.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### First, rf regression with [all features but MAR and BGI]. Runs back to 1980 or so 

In [3]:
df.columns

Index(['Date', 'Open', 'Close', 'monthly_return', 'period of downturn',
       'Recessions', 'INDPRO', 'yc_date', 'YIELD_CURVE', 'UNEMPLOYMENT',
       'BGI_diff_from_2_month_high', 'KO_diff_from_2_month_high',
       'KR_diff_from_2_month_high', 'MAR_diff_from_2_month_high',
       'PPG_diff_from_2_month_high', 'SHW_diff_from_2_month_high',
       'unemployment_diff_from_6_month_low', 'INDPRO_diff_from_6_month_high'],
      dtype='object')

In [4]:
REL_VARS = ['YIELD_CURVE','KO_diff_from_2_month_high',
       'KR_diff_from_2_month_high',
       'PPG_diff_from_2_month_high', 'SHW_diff_from_2_month_high',
       'unemployment_diff_from_6_month_low', 'INDPRO_diff_from_6_month_high']

df_train = df[df['Date']<pd.to_datetime('2005-01-01')]
df_test = df[df['Date']>=pd.to_datetime('2005-01-01')]

df_train.dropna(subset=REL_VARS,inplace=True)
df_test.dropna(subset=REL_VARS,inplace=True)

x_train = df_train[REL_VARS]
x_test = df_test[REL_VARS]

y_train = df_train['monthly_return']
y_test = df_test['monthly_return']


rf = RandomForestRegressor(n_estimators=2500,max_depth=4,max_features='sqrt')
rf.fit(x_train,y_train)

print(rf.score(x_train,y_train))

df_test['pred'] = rf.predict(x_test)

print(rf.score(x_test,y_test))

df_test.to_csv('testset_returns_regression_1980.csv',index=False)

/Users/samuelholden/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/Users/samuelholden/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


0.22706289849224193
-0.031849495860119204


/Users/samuelholden/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


#### Classification from 1980

In [5]:
REL_VARS = ['YIELD_CURVE','KO_diff_from_2_month_high',
       'KR_diff_from_2_month_high',
       'PPG_diff_from_2_month_high', 'SHW_diff_from_2_month_high',
       'unemployment_diff_from_6_month_low', 'INDPRO_diff_from_6_month_high']

df_train = df[df['Date']<pd.to_datetime('2005-01-01')]
df_test = df[df['Date']>=pd.to_datetime('2005-01-01')]

df_train.dropna(subset=REL_VARS,inplace=True)
df_test.dropna(subset=REL_VARS,inplace=True)

x_train = df_train[REL_VARS]
x_test = df_test[REL_VARS]

y_train = df_train['period of downturn']
y_test = df_test['period of downturn']


rf = RandomForestClassifier(n_estimators=2500,max_depth=4,max_features='sqrt')
rf.fit(x_train,y_train)

print(rf.score(x_train,y_train))
print(rf.score(x_test,y_test))

df_train['pred'] = rf.predict_proba(x_train)[:,1]

df_test['pred'] = rf.predict_proba(x_test)[:,1]

df_train.to_csv('trainset_returns_classification_1980.csv',index=False)
df_test.to_csv('testset_returns_classification_1980.csv',index=False)

/Users/samuelholden/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/Users/samuelholden/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


0.9191919191919192
0.9204545454545454


/Users/samuelholden/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/samuelholden/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


#### Let us try IND_PRO, UNEMPLOYMENT and Coca-Cola (from 1962) reg and classification below

In [6]:
REL_VARS = ['KO_diff_from_2_month_high',
       'unemployment_diff_from_6_month_low', 'INDPRO_diff_from_6_month_high']

df_train = df[df['Date']<pd.to_datetime('2005-01-01')]
df_test = df[df['Date']>=pd.to_datetime('2005-01-01')]

df_train.dropna(subset=REL_VARS,inplace=True)
df_test.dropna(subset=REL_VARS,inplace=True)

x_train = df_train[REL_VARS]
x_test = df_test[REL_VARS]

y_train = df_train['monthly_return']
y_test = df_test['monthly_return']


rf = RandomForestRegressor(n_estimators=2500,max_depth=4,max_features='sqrt')
rf.fit(x_train,y_train)

print(rf.score(x_train,y_train))

df_test['pred'] = rf.predict(x_test)

print(rf.score(x_test,y_test))

df_test.to_csv('testset_returns_regression_1962.csv',index=False)

/Users/samuelholden/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/samuelholden/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0.12544402312305913
-0.020166294236235283


/Users/samuelholden/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [7]:
REL_VARS = ['KO_diff_from_2_month_high',
       'unemployment_diff_from_6_month_low', 'INDPRO_diff_from_6_month_high']

df_train = df[df['Date']<pd.to_datetime('2005-01-01')]
df_test = df[df['Date']>=pd.to_datetime('2005-01-01')]

df_train.dropna(subset=REL_VARS,inplace=True)
df_test.dropna(subset=REL_VARS,inplace=True)

x_train = df_train[REL_VARS]
x_test = df_test[REL_VARS]

y_train = df_train['period of downturn']
y_test = df_test['period of downturn']


rf = RandomForestClassifier(n_estimators=2500,max_depth=4,max_features='sqrt')
rf.fit(x_train,y_train)

print(rf.score(x_train,y_train))
print(rf.score(x_test,y_test))

df_train['pred'] = rf.predict_proba(x_train)[:,1]

df_test['pred'] = rf.predict_proba(x_test)[:,1]

df_train.to_csv('trainset_returns_classification_1962.csv',index=False)
df_test.to_csv('testset_returns_classification_1962.csv',index=False)

/Users/samuelholden/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/samuelholden/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0.8330097087378641
0.9204545454545454


/Users/samuelholden/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/samuelholden/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


#### 1980 was better (more features available in the dataset)

#### Let us visualize feature importance in the 1980 example

In [8]:
REL_VARS = ['YIELD_CURVE',
        'KO_diff_from_2_month_high',
       'KR_diff_from_2_month_high',
       'PPG_diff_from_2_month_high', 
        'SHW_diff_from_2_month_high',
       'unemployment_diff_from_6_month_low', 
        'INDPRO_diff_from_6_month_high']

df_train = df[df['Date']<pd.to_datetime('2005-01-01')]
df_test = df[df['Date']>=pd.to_datetime('2005-01-01')]

df_train.dropna(subset=REL_VARS,inplace=True)
df_test.dropna(subset=REL_VARS,inplace=True)

x_train = df_train[REL_VARS]
x_test = df_test[REL_VARS]

y_train = df_train['period of downturn']
y_test = df_test['period of downturn']


rf = RandomForestClassifier(n_estimators=2500,max_depth=4,max_features='sqrt')
rf.fit(x_train,y_train)

print("let's see the feature importances:")
print(rf.feature_importances_)

/Users/samuelholden/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/Users/samuelholden/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


let's see the feature importances:
[0.41143017 0.09555708 0.07509091 0.11201642 0.09725733 0.10565904
 0.10298906]


#### Let's get rid of the bottom three features and rerun (results saved this time.)

In [9]:
REL_VARS = ['YIELD_CURVE',
       'PPG_diff_from_2_month_high',
       'unemployment_diff_from_6_month_low', 
        'INDPRO_diff_from_6_month_high']

df_train = df[df['Date']<pd.to_datetime('2005-01-01')]
df_test = df[df['Date']>=pd.to_datetime('2005-01-01')]
print(df_test.shape)

df_train.dropna(subset=REL_VARS,inplace=True)
df_test.dropna(subset=REL_VARS,inplace=True)

x_train = df_train[REL_VARS]
x_test = df_test[REL_VARS]

y_train = df_train['period of downturn']
y_test = df_test['period of downturn']


rf = RandomForestClassifier(n_estimators=2500,max_depth=4,max_features='sqrt')
rf.fit(x_train,y_train)

print("let's see the feature importances:")
print(rf.feature_importances_)

df_train['pred'] = rf.predict_proba(x_train)[:,1]

df_test['pred'] = rf.predict_proba(x_test)[:,1]

#df_train.to_csv('trainset_returns_classification_1980.csv',index=False)
#df_test.to_csv('testset_returns_classification_1980.csv',index=False)

(176, 18)


/Users/samuelholden/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/samuelholden/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


let's see the feature importances:
[0.5477267  0.16764122 0.14423534 0.14039675]


/Users/samuelholden/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/samuelholden/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


#### Ok so yield curve, PPG, unemployment and Industrial Prod have some signal. I think a sequence-based modeling technique will allow us to uncover the point of "first-trouble" that holds a lot of the signal in the three features of lesser importance according to RF. Pretty obvious when you consider how industrial production and unemployment continue to deteriorate  for a few months after financial markets have begun their recovery. Possibly sarimax, but likely need interactions. Let's see if RNN plays nice with our small dataset.